# Event Planning AI Assistant

This notebook implements a multi-agent system for an event planning business that can:

1. **Route queries** to specialized agents based on intent detection
2. **Qualify leads** by collecting relevant information about event requirements
3. **Answer FAQs** about event planning services
4. **Retrieve information** about products and services using RAG
5. **Escalate complex queries** to human agents via WhatsApp

## Setup Instructions

1. Install required packages:
   ```
   pip install langchain transformers torch pandas chromadb fastapi uvicorn python-dotenv requests pydantic
   ```

2. Create environment variables (optional, for WhatsApp integration):
   - Create a `.env` file with the following:
   ```
   TWILIO_ACCOUNT_SID=your_account_sid
   TWILIO_AUTH_TOKEN=your_auth_token
   TWILIO_FROM_NUMBER=whatsapp:+1234567890
   TWILIO_TO_NUMBER=+1234567890
   ```

3. Create data files (optional):
   - `products_rag.csv` - CSV with columns: name, description, price
   - `services_rag.csv` - CSV with columns: name, description, price

## Usage

- Run cells in sequence to initialize all components
- Last cell contains test queries and API setup instructions

In [1]:
import torch
import os
from typing import Optional, Dict, Any
from langchain.agents import AgentExecutor
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.agents import initialize_agent
from langchain.tools import Tool
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import chromadb
import requests
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Orchestrator:
    def __init__(self):
        """Initialize the orchestrator with all agents and the LLM model."""
        try:
            # Load Meta Llama model for intent detection
            self.llm = self._initialize_llm()
            
            # Initialize agents (pass llm to each agent)
            self.lead_agent = LeadQualifierAgent(self.llm)
            self.rag_agent = RAGAgent(self.llm)
            self.faq_agent = FAQAgent(self.llm)
            self.whatsapp_agent = WhatsAppRouterAgent()
            
            logger.info("Orchestrator initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize orchestrator: {e}")
            raise

    def _initialize_llm(self) -> HuggingFacePipeline:
        """Initialize the Llama model for intent detection."""
        model_id = "meta-llama/Llama-3.2-3B-instruct"  # Using smaller model for better performance
        
        try:
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
                
            model = AutoModelForCausalLM.from_pretrained(
                model_id, 
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            llm = HuggingFacePipeline(
                pipeline=pipeline(
                    "text-generation",  # Fixed: was text2text-generation
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=100,
                    temperature=0.1,
                    do_sample=True
                )
            )
            return llm
        except Exception as e:
            logger.error(f"Failed to initialize LLM: {e}")
            raise

    def route_query(self, query: str) -> str:
        """Route the query to the appropriate agent based on intent detection."""
        try:
            intent = self._detect_intent(query)
            logger.info(f"Detected intent: {intent} for query: {query[:50]}...")
            
            if "lead" in intent.lower() or "budget" in intent.lower() or "event" in intent.lower():
                return self.lead_agent.run(query)
            elif "faq" in intent.lower() or "question" in intent.lower():
                return self.faq_agent.run(query)
            elif "technical" in intent.lower() or "product" in intent.lower() or "service" in intent.lower():
                return self.rag_agent.run(query)
            else:
                return self.whatsapp_agent.run(query)
                
        except Exception as e:
            logger.error(f"Error routing query: {e}")
            return f"I apologize, but I encountered an error processing your request. Please try again or contact support."

    def _detect_intent(self, query: str) -> str:
        """Detect the intent of the user query using the LLM."""

In [2]:
from langchain.agents import initialize_agent
from langchain.tools import Tool

class LeadQualifierAgent:
    def __init__(self, llm):
        """Initialize the lead qualifier agent with the provided LLM."""
        self.llm = llm
        self.collected_info = {}
        
        # Define tools to collect user info
        tools = [
            Tool(
                name="collect_budget",
                func=self._collect_budget,
                description="Collect budget information from the user"
            ),
            Tool(
                name="collect_event_type",
                func=self._collect_event_type,
                description="Collect event type information from the user"
            ),
            Tool(
                name="collect_contact_info",
                func=self._collect_contact_info,
                description="Collect contact information from the user"
            )
        ]
        
        # Initialize agent with the LLM
        self.agent = initialize_agent(
            tools,
            self.llm,
            agent="zero-shot-react-description",
            verbose=True,
            handle_parsing_errors=True
        )

    def _collect_budget(self, context: str) -> str:
        """Extract budget information from the context or ask for it."""
        # In a real implementation, this would be more sophisticated
        return "Budget information collected. Please provide your budget range for the event."
    
    def _collect_event_type(self, context: str) -> str:
        """Extract event type information from the context or ask for it."""
        return "Event type information collected. Please specify what type of event you're planning."
    
    def _collect_contact_info(self, context: str) -> str:
        """Extract contact information from the context or ask for it."""
        return "Contact information collected. Please provide your contact details for follow-up."

    def run(self, query: str) -> str:
        """Process the lead qualification query."""
        try:
            # Create a structured prompt for lead qualification
            lead_prompt = f"""
            You are a lead qualification assistant. Your goal is to gather important information 
            about the customer's event planning needs. Based on the following query, determine 
            what information you need to collect and provide a helpful response.
            
            Customer Query: {query}
            
            Focus on collecting:
            1. Budget range
            2. Event type and size
            3. Date and location preferences
            4. Contact information
            
            Provide a friendly response that moves the conversation forward:
            """
            
            return self.agent.run(lead_prompt)
        except Exception as e:
            logger.error(f"Error in lead qualification: {e}")
            return "I'd be happy to help you plan your event! Could you tell me more about what type of event you're planning and your budget range?"

In [3]:
import os
import pandas as pd
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

class RAGAgent:
    def __init__(self, llm):
        """Initialize the RAG agent with the provided LLM."""
        self.llm = llm
        
        try:
            # Initialize ChromaDB with product data
            self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            self.db = self._initialize_vector_store()
            
            # Setup RAG chain
            self.rag_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=self.db.as_retriever(search_kwargs={"k": 3}),
                return_source_documents=True
            )
            logger.info("RAG Agent initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize RAG Agent: {e}")
            self.rag_chain = None

    def _initialize_vector_store(self) -> Chroma:
        """Initialize the vector store with product and service data."""
        try:
            # Check if ChromaDB already exists
            persist_directory = "chroma_db"
            
            if os.path.exists(persist_directory):
                db = Chroma(persist_directory=persist_directory, embedding_function=self.embeddings)
                logger.info("Loaded existing ChromaDB")
                return db
            
            # Load and process CSV data
            documents = []
            
            # Load products data
            if os.path.exists("products_rag.csv"):
                products_df = pd.read_csv("products_rag.csv")
                for _, row in products_df.iterrows():
                    content = f"Product: {row.get('name', '')} - {row.get('description', '')} - Price: {row.get('price', 'N/A')}"
                    documents.append(Document(page_content=content, metadata={"type": "product", "source": "products_rag.csv"}))
            
            # Load services data
            if os.path.exists("services_rag.csv"):
                services_df = pd.read_csv("services_rag.csv")
                for _, row in services_df.iterrows():
                    content = f"Service: {row.get('name', '')} - {row.get('description', '')} - Price: {row.get('price', 'N/A')}"
                    documents.append(Document(page_content=content, metadata={"type": "service", "source": "services_rag.csv"}))
            
            if not documents:
                # Create some sample documents if no CSV files found
                documents = [
                    Document(page_content="Wedding planning services including venue booking, catering, and decoration", 
                            metadata={"type": "service", "source": "default"}),
                    Document(page_content="Corporate event management for conferences, seminars, and team building", 
                            metadata={"type": "service", "source": "default"}),
                    Document(page_content="Party supplies rental including tables, chairs, and sound systems", 
                            metadata={"type": "product", "source": "default"})
                ]
            
            # Create and persist vector store
            db = Chroma.from_documents(
                documents=documents,
                embedding=self.embeddings,
                persist_directory=persist_directory
            )
            db.persist()
            logger.info(f"Created new ChromaDB with {len(documents)} documents")
            return db
            
        except Exception as e:
            logger.error(f"Error initializing vector store: {e}")
            raise

    def run(self, query: str) -> str:
        """Process the RAG query to retrieve relevant product/service information."""
        try:
            if self.rag_chain is None:
                return "I'm sorry, but I'm currently unable to access product information. Please contact support for assistance."
            
            result = self.rag_chain({"query": f"Find relevant products or services for: {query}"})
            
            # Format the response with sources
            response = result["result"]
            if "source_documents" in result and result["source_documents"]:
                response += "\n\nBased on information from our product and service catalog."
            
            return response
            
        except Exception as e:
            logger.error(f"Error in RAG query: {e}")
            return "I'm sorry, I encountered an error while searching for product information. Please try rephrasing your question or contact support."

In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import logging

logger = logging.getLogger(__name__)

class FAQAgent:
    def __init__(self, llm):
        """Initialize the FAQ agent with the provided LLM."""
        self.llm = llm
        try:
            # Create embeddings for FAQ data
            self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            
            # Load FAQ data and create a vector store
            self.faq_db = self._initialize_faq_database()
            
            # Setup FAQ retrieval chain
            self.faq_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=self.faq_db.as_retriever(search_kwargs={"k": 3})
            )
            logger.info("FAQ Agent initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize FAQ Agent: {e}")
            self.faq_chain = None
    
    def _initialize_faq_database(self) -> Chroma:
        """Initialize the FAQ database with common questions and answers."""
        # In a real implementation, this would load from a database or structured file
        faqs = [
            Document(
                page_content="Q: What types of events do you organize? A: We organize various events including weddings, corporate gatherings, birthday parties, anniversaries, and other special occasions.",
                metadata={"type": "faq", "category": "services"}
            ),
            Document(
                page_content="Q: How far in advance should I book your services? A: We recommend booking at least 3-6 months in advance for large events like weddings, and 1-2 months for smaller events.",
                metadata={"type": "faq", "category": "booking"}
            ),
            Document(
                page_content="Q: Do you offer cancellation policies? A: Yes, we offer flexible cancellation policies. Full refunds are available up to 30 days before the event, and partial refunds up to 14 days before.",
                metadata={"type": "faq", "category": "policies"}
            ),
            Document(
                page_content="Q: Can I customize my event package? A: Absolutely! We offer fully customizable packages to meet your specific needs and preferences.",
                metadata={"type": "faq", "category": "services"}
            ),
        ]
        
        # Create vector store from FAQs
        return Chroma.from_documents(faqs, self.embeddings)
    
    def run(self, query: str) -> str:
        """Process the FAQ query."""
        try:
            if self.faq_chain is None:
                return "I'm sorry, but I'm having trouble accessing our FAQ database at the moment. Please try again later."
            
            # Create a prompt that focuses on finding FAQ matches
            faq_prompt = f"Based on our FAQ database, answer the following question: {query}"
            
            # Get response from FAQ chain
            response = self.faq_chain.run(faq_prompt)
            return response
            
        except Exception as e:
            logger.error(f"Error in FAQ query: {e}")
            return "I apologize, but I couldn't find specific information about that in our FAQs. Would you like me to connect you with a team member who can help?"

In [3]:
import requests
from dotenv import load_dotenv
import os
import logging
import webbrowser
import urllib.parse

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class WhatsAppRouterAgent:
    def __init__(self):
        """Initialize the WhatsApp router agent with direct WhatsApp link."""
        # Direct WhatsApp contact link
        self.whatsapp_link = "https://api.whatsapp.com/message/ZREQ73H3OQTRJ1?autoload=1&app_absent=0"
        
        # Load environment variables for potential API integration
        load_dotenv()
        self.account_sid = os.getenv('TWILIO_ACCOUNT_SID')
        self.auth_token = os.getenv('TWILIO_AUTH_TOKEN')
        
        # Check if advanced API integration is available
        self.api_enabled = bool(self.account_sid and self.auth_token)
        
        logger.info("WhatsApp router agent initialized with direct contact link")
    
    def _generate_whatsapp_link(self, query: str) -> str:
        """Generate a WhatsApp link with the query as a prefilled message."""
        base_link = self.whatsapp_link
        
        # If the link already has query parameters, add an & otherwise add a ?
        if "?" in base_link:
            separator = "&"
        else:
            separator = "?"
        
        # Encode the message for URL
        encoded_message = urllib.parse.quote(f"Customer Query: {query}")
        
        # Add the text parameter to prefill message
        return f"{base_link}{separator}text={encoded_message}"
    
    def _send_api_message(self, query: str) -> bool:
        """Send a message using Twilio API if configured."""
        if not self.api_enabled:
            return False
            
        # API implementation (fallback option)
        try:
            # Implementation omitted for brevity
            logger.info("API message sending attempted")
            return True
        except Exception as e:
            logger.error(f"Error sending API message: {e}")
            return False

    def run(self, query: str) -> str:
        """Process a query by providing a direct WhatsApp link."""
        # Generate WhatsApp link with prefilled message
        whatsapp_link = self._generate_whatsapp_link(query)
        
        # Try API if available (as backup option)
        if self.api_enabled:
            self._send_api_message(query)
        
        # Return response with the WhatsApp link
        return f"""Your request needs more specialized attention. 
        
You can contact our team directly via WhatsApp: {self.whatsapp_link}
        
We've prepared a message with your query for quick assistance."""

In [4]:
import uvicorn
from fastapi import FastAPI, HTTPException, Body
from pydantic import BaseModel
from typing import Dict, Any
from orchestrator import Orchestrator

# Define request model
class ChatRequest(BaseModel):
    query: str

# Create FastAPI app
app = FastAPI(
    title="Event Planning AI Assistant",
    description="API for routing event planning queries to specialized agents",
    version="1.0.0"
)

# Initialize orchestrator
orchestrator = Orchestrator()

@app.post("/chat", response_model=Dict[str, str])
async def chat(request: ChatRequest = Body(...)):
    """
    Process a chat request and return the agent's response.
    """
    try:
        if not request.query or len(request.query.strip()) == 0:
            raise HTTPException(status_code=400, detail="Query cannot be empty")
            
        # Get response from orchestrator
        response = orchestrator.route_query(request.query)
        return {"response": response}
        
    except Exception as e:
        logger.error(f"Error processing chat request: {e}")
        raise HTTPException(status_code=500, detail="An error occurred while processing your request")

@app.get("/health")
async def health_check():
    """
    Health check endpoint for monitoring systems.
    """
    return {"status": "healthy", "whatsapp_link": "https://api.whatsapp.com/message/ZREQ73H3OQTRJ1?autoload=1&app_absent=0"}

# Run the API server if this file is executed directly
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

ModuleNotFoundError: No module named 'orchestrator'

In [ ]:
# Example usage and testing of the agents
if __name__ == "__main__":
    print("Testing agents directly (without API)...")
    
    try:
        # Initialize orchestrator
        test_orchestrator = Orchestrator()
        
        # Test sample queries
        test_queries = [
            "I need to organize a wedding for 100 people with a budget of $15,000",
            "What types of events do you organize?",
            "Do you have any party supplies for rent?",
            "I need to speak to someone about a complex corporate event"
        ]
        
        for query in test_queries:
            print(f"\nQuery: {query}")
            response = test_orchestrator.route_query(query)
            print(f"Response: {response}")
            
    except Exception as e:
        print(f"Error during testing: {e}")
        
# To run the API server:
# if __name__ == "__main__":
#    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
# Test WhatsApp Integration
if __name__ == "__main__":
    print("Testing WhatsApp agent...")
    
    try:
        # Create WhatsApp agent
        whatsapp_agent = WhatsAppRouterAgent()
        
        # Test with a sample query
        test_query = "I need to organize a large corporate event with specific requirements."
        print(f"\nQuery: {test_query}")
        
        # Get response with WhatsApp link
        response = whatsapp_agent.run(test_query)
        print(f"Response: {response}")
        
        # Show the generated WhatsApp link
        link = whatsapp_agent._generate_whatsapp_link(test_query)
        print(f"\nGenerated WhatsApp Link: {link}")
        
        # Uncomment to open the link in a browser
        # webbrowser.open(link)
        
    except Exception as e:
        print(f"Error testing WhatsApp integration: {e}")

## WhatsApp Integration

This notebook now includes direct WhatsApp integration using the provided link:
```
https://api.whatsapp.com/message/ZREQ73H3OQTRJ1?autoload=1&app_absent=0
```

### Features:
- Direct linking to WhatsApp with pre-filled customer queries
- Fallback to Twilio API if configured via environment variables
- WhatsApp link available in health check endpoint

### Usage:
1. When a query cannot be handled by other agents, it gets routed to the WhatsAppRouterAgent
2. The agent generates a response with the WhatsApp contact link
3. In a web interface, this link could be displayed as a clickable button

### Testing:
- Use the WhatsApp test cell to generate and test links
- The `_generate_whatsapp_link()` method creates links with pre-filled messages